<a href="https://colab.research.google.com/github/ruih12/ec601-team/blob/sprint4/sprint4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import subprocess

# 克隆项目
repo_url = "https://github.com/AliaksandrSiarohin/first-order-model.git"
repo_dir = "/content/first-order-model"

if not os.path.exists(repo_dir):
    print("克隆项目...")
    subprocess.run(["git", "clone", repo_url, repo_dir])
else:
    print("项目已存在！")

# 切换到项目目录
os.chdir(repo_dir)

# 安装依赖
print("安装依赖...")
subprocess.run(["pip", "install", "-r", "requirements.txt"])

# 创建必要的文件夹
os.makedirs("aligned_faces", exist_ok=True)
os.makedirs("reference_videos", exist_ok=True)
os.makedirs("generated_videos", exist_ok=True)


项目已存在！
安装依赖...


In [4]:
import requests

# 下载预训练权重文件
checkpoint_url = "https://drive.google.com/uc?id=1PyMAmvkoxI0bR8hfdJrQC5kqE_tKjH1Q"
checkpoint_path = "checkpoints/vox-cpk.pth.tar"
os.makedirs("checkpoints", exist_ok=True)

if not os.path.exists(checkpoint_path):
    print("下载预训练权重文件...")
    subprocess.run(["gdown", checkpoint_url, "-O", checkpoint_path])
else:
    print("预训练权重文件已存在！")

# 下载配置文件
config_url = "https://github.com/AliaksandrSiarohin/first-order-model/raw/master/config/vox-256.yaml"
config_path = "config/vox-256.yaml"
os.makedirs("config", exist_ok=True)

if not os.path.exists(config_path):
    print("下载配置文件...")
    response = requests.get(config_url)
    with open(config_path, "wb") as f:
        f.write(response.content)
    print("配置文件已成功下载！")
else:
    print("配置文件已存在！")


预训练权重文件已存在！
配置文件已存在！


In [5]:
from google.colab import files

# 上传静态图像
print("请上传静态图像（JPG/PNG 格式）到 aligned_faces 文件夹...")
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, os.path.join("aligned_faces", filename))

# 上传参考视频
print("请上传参考视频（MP4 格式）到 reference_videos 文件夹...")
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, os.path.join("reference_videos", filename))


请上传静态图像（JPG/PNG 格式）到 aligned_faces 文件夹...


Saving 1953_Marilyn_Monroe.jpg to 1953_Marilyn_Monroe.jpg
Saving Marilyn_Monroe,_Epoca.jpg to Marilyn_Monroe,_Epoca.jpg
Saving Marilyn_Monroe,_Korea,_1954_cropped.jpg to Marilyn_Monroe,_Korea,_1954_cropped.jpg
Saving Marilyn_Monroe,_Photoplay_1953.jpg to Marilyn_Monroe,_Photoplay_1953.jpg
Saving Marilyn_Monroe,_The_Prince_and_the_Showgirl,_2.jpg to Marilyn_Monroe,_The_Prince_and_the_Showgirl,_2.jpg
Saving Marilyn_Monroe_1952.jpg to Marilyn_Monroe_1952.jpg
Saving Marilyn_Monroe_1953_crop.jpg to Marilyn_Monroe_1953_crop.jpg
Saving Marilyn_Monroe_1953_crop_2.jpg to Marilyn_Monroe_1953_crop_2.jpg
Saving Marilyn_Monroe_1961.jpg to Marilyn_Monroe_1961.jpg
Saving Marilyn_Monroe_in_1952_TFA.jpg to Marilyn_Monroe_in_1952_TFA.jpg
Saving Marilyn_Monroe_in_Gentlemen_Prefer_Blondes_trailer.jpg to Marilyn_Monroe_in_Gentlemen_Prefer_Blondes_trailer.jpg
Saving Marilyn_Monroe_in_How_to_Marry_a_Millionaire.jpg to Marilyn_Monroe_in_How_to_Marry_a_Millionaire.jpg
Saving Marilyn_Monroe_wax_figure.jpg to Ma

Saving reference video.mp4 to reference video.mp4


In [6]:
# 下载示例静态图像
example_image_url = "https://upload.wikimedia.org/wikipedia/commons/3/37/Marilyn_Monroe_-_publicity.JPG"
example_image_path = "aligned_faces/example.jpg"
if not os.path.exists(example_image_path):
    response = requests.get(example_image_url)
    with open(example_image_path, "wb") as f:
        f.write(response.content)
    print("示例静态图像已下载！")

# 下载示例参考视频
example_video_url = "https://sample-videos.com/video123/mp4/720/big_buck_bunny_720p_1mb.mp4"
example_video_path = "reference_videos/example.mp4"
if not os.path.exists(example_video_path):
    response = requests.get(example_video_url)
    with open(example_video_path, "wb") as f:
        f.write(response.content)
    print("示例参考视频已下载！")


In [7]:
import os

# 创建目标文件夹
checkpoints_dir = "/content/first-order-model/checkpoints"
os.makedirs(checkpoints_dir, exist_ok=True)
print(f"目标文件夹已创建：{checkpoints_dir}")


目标文件夹已创建：/content/first-order-model/checkpoints


In [8]:
from demo import load_checkpoints

# 加载模型
config_path = "config/vox-256.yaml"
checkpoint_path = "checkpoints/vox-cpk.pth.tar"

generator, kp_detector = load_checkpoints(config_path=config_path, checkpoint_path=checkpoint_path)
print("模型加载成功！")


/usr/local/lib/python3.10/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


模型加载成功！


In [ ]:
import os
import gc
from skimage import io
import numpy as np
from demo import make_animation
import imageio.v2 as imageio
import cv2

# 定义文件夹路径
aligned_faces_dir = "aligned_faces"
reference_videos_dir = "reference_videos"
output_videos_dir = "generated_videos"

# 创建输出文件夹
os.makedirs(output_videos_dir, exist_ok=True)

# 遍历每个静态图像和参考视频
for source_image_name in os.listdir(aligned_faces_dir):
    source_image_path = os.path.join(aligned_faces_dir, source_image_name)

    # 检查静态图像是否为有效文件
    if not source_image_name.lower().endswith((".jpg", ".png")):
        print(f"跳过非图像文件: {source_image_name}")
        continue

    for reference_video_name in os.listdir(reference_videos_dir):
        reference_video_path = os.path.join(reference_videos_dir, reference_video_name)

        # 检查参考视频是否为有效文件
        if not reference_video_name.lower().endswith(".mp4"):
            print(f"跳过非视频文件: {reference_video_name}")
            continue

        try:
            print(f"处理: 静态图像={source_image_path}, 视频={reference_video_path}")

            # 加载静态图像
            source_image = io.imread(source_image_path)
            source_image = np.array(source_image)

            # 使用 OpenCV 逐帧读取参考视频
            cap = cv2.VideoCapture(reference_video_path)
            driving_video = []
            frame_count = 0

            while True:
                ret, frame = cap.read()
                if not ret or frame_count > 150:  # 限制处理帧数（例如最多处理 150 帧）
                    break
                driving_video.append(frame)
                frame_count += 1
            cap.release()

            if not driving_video:
                print(f"视频读取失败或为空: {reference_video_path}")
                continue

            # 动画生成
            output_video_path = os.path.join(output_videos_dir, f"{source_image_name.split('.')[0]}_{reference_video_name.split('.')[0]}.mp4")
            predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

            # 保存生成的视频
            imageio.mimsave(output_video_path, [np.array(frame) for frame in predictions], fps=30)
            print(f"生成的视频已保存到: {output_video_path}")

            # 手动释放内存
            del source_image, driving_video, predictions
            gc.collect()

        except Exception as e:
            print(f"处理失败: 静态图像={source_image_path}, 视频={reference_video_path}, 错误: {e}")


处理: 静态图像=aligned_faces/Marilyn_Monroe_1961.jpg, 视频=reference_videos/reference video.mp4
